# Clase 14: Manejo de Valores Faltantes

**MDS7202: Laboratorio de Programación Científica para Ciencia de Datos**

**Profesor: Matías Rojas**


## Roadmap


<img src='https://i.ibb.co/4tHWJRQ/etapas.png'/>


## Objetivos de la Clase

- Entender la complejidad de tener datos faltantes para los modelos.
- Entender los distintos tipos de datos faltantes.
- Explorar las opciones para solucionar el problema de datos faltantes.
- Integrar los conocimientos adquiridos al pipeline.

## Retomando nuestro Pipeline


Un pipeline es una lista de tuplas. 

- La lista contiene todos los pasos que se efectuan desde la entrada hasta la salida del pipeline
- Cata tupla de la lista representa un subproceso del pipeline. Este debe estar compuesto por un nombre u la clase que corresponda.

![Pipeline](https://i.ibb.co/bzkNpNS/pipeline.png)

Por ejemplo, para la imágen anterior, el pipeline sería: 

```python
pipe = Pipeline([('scaling', Scaler()),
                 ('dimensionality_reduction', DimReductor()),
                 ('predictive_model', Model())])
```

---

### Dataset - House Pricing


![House Pricing](https://storage.googleapis.com/kaggle-competitions/kaggle/5407/media/housesbanner.png)

Fuente: https://www.kaggle.com/c/house-prices-advanced-regression-techniques

El dataset **`house pricing`** consiste en 80 variables (79 variables explicativas más una variable objetivo) que describen aspectos fundamentales de hogares residenciales en la ciudad de *Ames, Iowa*. 

La variable objetivo es el precio final de cada hogar (regresión)

In [ ]:
import pandas as pd

df = pd.read_csv("https://raw.githubusercontent.com/maranedah/MDS7202/main/clases/Clase%2014%20-%20Manejo%20de%20Valores%20Faltantes/train.csv", index_col="Id")
df.head()

In [ ]:
cat_cols = [
    "MSZoning",
    "Street",
    "Alley",
    "LandContour",
    "LotConfig",
    "Neighborhood",
    "Condition1",
    "Condition2",
    "BldgType",
    "HouseStyle",
    "RoofStyle",
    "RoofMatl",
    "Exterior1st",
    "Exterior2nd",
    "MasVnrType",
    "Foundation",
    "Heating",
    "CentralAir",
    "Electrical",
    "GarageType",
    "PavedDrive",
    "MiscFeature",
    "SaleType",
    "SaleCondition",
]

ordinal_cols = [
    "LotShape",
    "Utilities",
    "LandSlope",
    "ExterQual",
    "ExterCond",
    "BsmtQual",
    "BsmtCond",
    "BsmtExposure",
    "BsmtFinType1",
    "BsmtFinType2",
    "HeatingQC",
    "KitchenQual",
    "Functional",
    "FireplaceQu",
    "GarageFinish",
    "GarageQual",
    "GarageCond",
    "PoolQC",
    "Fence",
]

# Adquieren las categorias de cada variable
ordinal_cat = [
    ["Reg", "IR1", "IR2", "IR3"],
    ["AllPub", "NoSewr", "NoSeWa", "ELO"],
    ["Gtl", "Mod", "Sev"],
    ["Po", "Fa", "TA", "Gd", "Ex"],
    ["Po", "Fa", "TA", "Gd", "Ex"],
    ["NA", "Po", "Fa", "TA", "Gd", "Ex"],
    ["NA", "Po", "Fa", "TA", "Gd", "Ex"],
    ["NA", "No", "Mn", "Av", "Gd"],
    ["NA", "Unf", "LwQ", "Rec", "BLQ", "ALQ", "GLQ"],
    ["NA", "Unf", "LwQ", "Rec", "BLQ", "ALQ", "GLQ"],
    ["Po", "Fa", "TA", "Gd", "Ex"],
    ["Po", "Fa", "TA", "Gd", "Ex"],
    ["Sal", "Sev", "Maj2", "Maj1", "Mod", "Min2", "Min1", "Typ"],
    ["NA", "Po", "Fa", "TA", "Gd", "Ex"],
    ["NA", "Unf", "RFn", "Fin"],
    ["NA", "Po", "Fa", "TA", "Gd", "Ex"],
    ["NA", "Po", "Fa", "TA", "Gd", "Ex"],
    ["NA", "Fa", "TA", "Gd", "Ex"],
    ["NA", "MnWw", "GdWo", "MnPrv", "GdPrv"],
]

num_cols = [
    "MSSubClass",
    "LotFrontage",
    "LotArea",
    "OverallQual",
    "OverallCond",
    "YearBuilt",
    "YearRemodAdd",
    "MasVnrArea",
    "BsmtFinSF1",
    "BsmtFinSF2",
    "BsmtUnfSF",
    "TotalBsmtSF",
    "1stFlrSF",
    "2ndFlrSF",
    "LowQualFinSF",
    "GrLivArea",
    "BsmtFullBath",
    "BsmtHalfBath",
    "FullBath",
    "HalfBath",
    "BedroomAbvGr",
    "KitchenAbvGr",
    "TotRmsAbvGrd",
    "Fireplaces",
    "GarageYrBlt",
    "GarageCars",
    "GarageArea",
    "WoodDeckSF",
    "OpenPorchSF",
    "EnclosedPorch",
    "3SsnPorch",
    "ScreenPorch",
    "PoolArea",
    "MiscVal",
    "MoSold",
    "YrSold",
]

In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, StandardScaler

In [ ]:
# Preprocesador Compuesto
preprocessing = ColumnTransformer(
    transformers=[
        ("standard_scaler", StandardScaler(), num_cols),
        (
            "category_one_hot",
            OneHotEncoder(sparse=False, handle_unknown="ignore"),
            cat_cols,
        ),
        ("ordinal_encoder", OrdinalEncoder(categories=ordinal_cat), ordinal_cols),
    ]
)

In [ ]:
preprocessing.transformers[0][1]

In [ ]:
pipe = Pipeline([("Preprocessing", preprocessing)])
pipe

## Exploración de Datos Faltantes



In [ ]:
# Instalamos el paquete usando conda o pip
import sys

import matplotlib.pyplot as plt

try:
    import missingno as msno
except ImportError as e: 
    !{sys.executable} -m pip install --upgrade missingno # descomentar si se usa pip
    
import missingno as msno

### Conteo de Datos no Nulos

In [ ]:
import plotly.express as px

conteo_nulos = (
    (len(df) - df.isna().sum())
    .to_frame("Obs")
    .reset_index()
    .rename(columns={"index": "Col"})
)

px.bar(
    conteo_nulos.sort_values("Obs"),
    x="Obs",
    y="Col",
    height=800,
    title="Cantidad de Datos no Nulos por Columna",
)

### Nullity Matrix

Permite visualizar los datos faltantes a traves de barras. 
Cada espacio en gris representa una observación y cada espacio en blanco simboliza un dato faltante.

In [ ]:
msno.matrix(df.iloc[:, :40])

In [ ]:
msno.matrix(df.iloc[:, 40:])

In [ ]:
df.loc[:, ["Fireplaces", "FireplaceQu"]]

> **Pregunta ❓**: ¿Qué representan las variables con mayor falta de información?




### Mapa de Calor


El mapa de calor se utiliza para identificar las correlaciones de nulidad (relación en la presencia de valores) entre las columnas. 

- Los valores cercanos a 1 indican que la presencia de valores nulos en una columna está correlacionada con la presencia de valores nulos en otra columna.

- Los valores cercanos a -1 indican que la presencia de valores nulos en una columna está inversamente correlacionada con la presencia de valores nulos en otra columna. En otras palabras, cuando los valores nulos están presentes en una columna, hay valores de datos presentes en la otra columna, y viceversa.

- Los valores cercanos a 0, indican que hay poca o ninguna relación entre la presencia de valores nulos en una columna en comparación con otra.

In [ ]:
import matplotlib.pyplot as plt

msno.heatmap(df)

## Manejo de valores faltantes

Por lo general, existen razones prácticas y conceptuales a tener en cuenta cuando se trabaja con valores faltantes. 

### Conceptual

**La falta de información introduce sesgos en los modelos de datos**, pues hace que las muestras obtenidas no sean representativas del fenómeno que se desea estudiar. Esto puede generar conclusiones sesgadas y puede llevar a tomar malas decisiones. 

### Práctica

**Los valores faltantes son incompatibles con algunos modelos de aprendizaje automático**, debido a que estos modelos son parte de la razón fundamental de analizar un fenómeno por medio de datos,es que se necesita comprender bien los mecanismos de manejo de este tipo de valores. 


### Tipos de datos Faltantes

Para el ejemplo, supongamos que efectuamos una encuesta a una población acerca de datos demográficos y de salarios.

|  | MCAR | MAR | MNAR |
|---|---|---|---|
| **Nombre** | Missing completely at random | Missing at Random | Missing not at Random |
| **Descripción** | Es el caso en que los datos faltantes son aleatorios y que esta falta no puede ser explicada por ninguna otra variable del dataset. <br>En este caso es util pensar que cualquier observación tiene una probabilidad de perderse y las que no están, simplemente se perdieron por azar. | Es el caso en que la perdida de datos está relacionado con alguna otra variable (no con si misma), pero la falta de información es aleatoria.<br>En este caso, la falta de información no es al azar.  | Es el caso en que los datos faltantes están relacionados con la variable en si. Es el peor tipo de dato faltante ya que lleva a calcular estadísticas descriptivas sesgadas. |
| **Ejemplo** | Una de las preguntas en la encuesta era la altura, cuya respuesta era opcional. Al revisar los resultados de la encuesta, notas que falta el 37% de los valores `altura`.<br><br>En este caso, no hay ningún patrón de perdida, es simplemente aleatorio.  | A partir de los 50 años, las personas tienden a dejar de entregar información respecto a su sueldo.<br><br>El patrón de perdida en este caso es la edad: desde los 50 años, la información sobre el sueldo tiende a faltar aleatoriamente. | Personas con sueldos mayores a los 1.250.000$ tienden a no informar su sueldo.<br><br>El patrón de perdida de información en este caso es el sueldo en si y no se relaciona con ninguna otra variable. |
| **Solución** | Eliminar las filas que no contengan los datos o imputar valores (por ejemplo, reemplazar los faltantes por la media, moda o mediana). | Imputar (en el ejemplo podría ser una regresión edad-sueldo) | Intentar recuperar esos datos de la fuente original del dataset, cruzar con otros datasets. |

### Análisis de Algunas Variables Según Tipo

In [ ]:
msno.matrix(df.iloc[:, :40])

In [ ]:
df.loc[:, ["Alley"]]

In [ ]:
df.loc[:, "Alley"].value_counts(dropna=False)

In [ ]:
# Existirá relación con alguna otra variable?
df.loc[:, ["Street", "Alley"]]

In [ ]:
# Eliminamos los nulos para ver si hay alguna relación
df.loc[:, ["Street", "Alley"]].dropna()

> **Pregunta ❓:** ¿Qué tipo de dato faltante es?

#### Alley (Callejón de Entrada)

Type of alley access to property

       Grvl	Gravel
       Pave	Paved
       NA 	No alley access

### LotFrontage

LotFrontage: Linear feet of street connected to property

In [ ]:
df.loc[:, ["LotFrontage"]]

In [ ]:
df.loc[df["LotFrontage"].isna(), :]

In [ ]:
msno.matrix(df.iloc[:, :40])

In [ ]:
msno.matrix(df.iloc[:, 40:])

### FireplaceQu

In [ ]:
df.loc[:, ["FireplaceQu"]]

In [ ]:
df.loc[:, ["Fireplaces", "FireplaceQu"]]

## Tratamiento de Datos Faltantes

### Eliminación (caso MCAR)

Es el método más sencillo, se conoce tambien como **list-wise deletion** y consiste en eliminar filas o columnas de un dataset que presenten datos faltantes. Se puede acceder a este tipo de tratamiento por medio de `.dropna()` objetos de Pandas.

En casos distintos (MAR o MNAR) su uso es contraindicado. Se recomiendan cuando el patrón de perdida de información observada (por ejemplo por medio de `mssingno`) es claramente aleatorio, y si además las variables con información faltante son 'pocas' y con 'pocos' valores faltantes. La definición de 'poco' varia en función del problema, pero una buena huerística puede ser inferior al 15% en variables de poca importancia.


In [ ]:
df.dropna(subset=["LotFrontage"])


### Imputación (caso MCAR-MAR)

Corresponde a las téncincas que permiten rellenar de información faltante por medio de estimaciones. Es usada comunmente cuando los datos faltantes son de caracter MAR.


#### Imputación Simple

Por lo general este tipo de imputación presenta un buen rendimiento empírico en tareas de ciencia de datos y es ampliamente recomendado. 

> **Nota:** aplicar este tipo de métodos puede afectar el calculo de varianzas y covarianzas. 

En `pandas` podemos aplicar este tipo de imputación por medio del método `.fillna()` ya sea entregando un valor precalculado (media, mediana, moda, etc...) o utilizando los argumentos `ffill` (usar el valor anterior)  y `bfill` (usar el valor siguiente). 
En `scikit-learn` por otra parte, podemos utilizar [`SimpleImputer`](https://scikit-learn.org/stable/modules/generated/sklearn.impute.SimpleImputer.html#sklearn.impute.SimpleImputer), el cual es una transformación, lo que hace este método compatible con los `Pipelines`.

Existen directrices a tener en cuenta al momento de tratar valores faltantes:

**1. Variables categóricas**:

    * Transformar valores faltantes en una nueva categoría.
    * Utilizar códificación Dummy en variables categóricas

In [ ]:
df["GarageType"]

In [ ]:
df["GarageType"].value_counts(dropna=False)

In [ ]:
df["GarageType"].isna()

In [ ]:
df[df["GarageType"].isna()]["GarageType"]

In [ ]:
garage_type = df["GarageType"].fillna("NoGarage")
garage_type.value_counts()

**2. Valores Ordinales**

    * Agregar la categoría de valor faltante como orden inicial o final en categorias ordinales.


In [ ]:
# ['Po', 'Fa', 'TA', 'Gd', 'Ex'],
df["GarageCond"]

In [ ]:
df["GarageCond"].value_counts(dropna=False)

In [ ]:
garage_cond = df["GarageCond"].fillna("NoGarage")
# Agregar después al Encoder NoGarage ['NoGarage', 'Po', 'Fa', 'TA', 'Gd', 'Ex'],
garage_cond.value_counts()

**3 Valores continuos**:

    * En MAR notar que imputar los datos puede cambiar la distribución original y por ende, llevar a malos resultados.

In [ ]:
df[df["LotFrontage"].isna()]

In [ ]:
df["LotFrontage"].describe()

En este caso, usaremos `SimpleImputer`.

La estrategia se define por el parámetro:

`strategy`, default=’mean’

    The imputation strategy.

        If “mean”, then replace missing values using the mean along each column. Can only be used with numeric data.

        If “median”, then replace missing values using the median along each column. Can only be used with numeric data.

        If “most_frequent”, then replace missing using the most frequent value along each column. Can be used with strings or numeric data. If there is more than one such value, only the smallest is returned.

        If “constant”, then replace missing values with fill_value. Can be used with strings or numeric data.


In [ ]:
from sklearn.impute import SimpleImputer

si = SimpleImputer(strategy="median")

imputed_lotfrontage = si.fit_transform(df.loc[:, ["LotFrontage"]])

imputed_lotfrontage = pd.DataFrame(imputed_lotfrontage, columns=["ImputedLotFrontage"])

imputed_lotfrontage

In [ ]:
imputed_lotfrontage.describe()

In [ ]:
import plotly.express as px

lf = pd.concat((df.loc[:, ["LotFrontage"]], imputed_lotfrontage))
px.histogram(lf, barmode="group")

#### Imputación Multivariada

Un problema de la imputación singular es que modela los datos como uno completo, sin considerar la incertidumbre inherente a todos los otros datos. Una solución para esto son los métodos de imputación múltiple.

Un método de inmputación multiple estima los valores faltantes a partir de otros. Puede ser tanto univariada (solo considerando la variable objetivo) como multivariada.

##### `IterativeImputer`

Un enfoque sofisticado es el que plantea `IterativeImputer`, el cuál intenta modelar los valores faltantes a partir como función de los otros valores. El algoritmo que ocupa es el siguiente:

En la primera iteración:
- Por cada columna con datos faltantes:
   - Se entrena un regresor que intenta predecir los valores de esa columna a partir de todas las otras. El regresor usa como target los valores no faltantes de la columna objetivo.
   - Se reemplazan los valores faltantes por los predichos por el regresor.
   
Luego, se repite `max_iter` cantidad de veces el mismo procedimiento, usando los valores originales y también los valores predichos en la etapa anterior. La idea de esto es que este procedimiento iterativo vaya "actualizando" hasta que los valores predichos convergan.

Su uso es experimental y se activa siguiendo la sintaxis:

```python
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
```

**Ejemplo**

Se utiliza el imputador iterativo para los estimadores de Random Forest y KNN.

**NOTA IMPORTANTE**: El siguiente código es solo de ejemplo y se asume que todas las variables son MAR, puede que no sea correcto imputar estas features.

In [ ]:
import numpy as np

only_numeric = df.select_dtypes(include=np.number).drop(
    columns=[
        "YearBuilt",
        "YearRemodAdd",
        "YrSold",
        "GarageYrBlt",
        "SalePrice",
        "MoSold",
        "MSSubClass",
    ]
)
only_numeric

##### IterativeImputer 

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

In [ ]:
msno.matrix(only_numeric)

In [ ]:
imputer = IterativeImputer(skip_complete=True, verbose=1, random_state=1)

RF_imputed = imputer.fit_transform(only_numeric)
RF_imputed = pd.DataFrame(RF_imputed, columns=only_numeric.columns)
RF_imputed

In [ ]:
msno.matrix(RF_imputed)

In [ ]:
lf_imputed = RF_imputed.loc[:, ["LotFrontage"]]
lf_imputed.columns = ["LotFrontageImputed"]

lf = pd.concat((df.loc[:, ["LotFrontage"]], lf_imputed))
px.histogram(lf, barmode="group", nbins=50)

In [ ]:
px.histogram(lf, barmode="group", nbins=50, histnorm="probability")

##### KNNImputer

Imputa usando k-Nearest Neighbors.

Los valores imputados se calculan según el parámetro `weights`:

- Si `weights=uniform`, se promedian los valores de los vecinos cercanos.
- Si `weights=distance`, se ponderan los valores de los vecinos cercanos según la distancia al punto.

<img src='https://i.ibb.co/Kjn0FWM/knn.png' width=400/> 

In [ ]:
from sklearn.impute import KNNImputer

# en este caso, simplemente se promedian los valores cercanos.
KNNimputer = KNNImputer(n_neighbors=2, weights="uniform")
KNN_imputed_data = KNNimputer.fit_transform(only_numeric)
KNN_imputed_data = pd.DataFrame(KNN_imputed_data, columns=only_numeric.columns)

In [ ]:
lf_imputed = KNN_imputed_data.loc[:, ["LotFrontage"]]
lf_imputed.columns = ["LotFrontageImputed"]

lf = pd.concat((df.loc[:, ["LotFrontage"]], lf_imputed))
px.histogram(lf, barmode="group", nbins=50)

### Incluyendo la imputación en nuestra Pipeline

Nota que ambos métodos pueden ser usados en un Pipeline.

In [ ]:
cat_cols = [
    "MSZoning",
    "Street",
    "Alley",
    "LandContour",
    "LotConfig",
    "Neighborhood",
    "Condition1",
    "Condition2",
    "BldgType",
    "HouseStyle",
    "RoofStyle",
    "RoofMatl",
    "Exterior1st",
    "Exterior2nd",
    "MasVnrType",
    "Foundation",
    "Heating",
    "CentralAir",
    "Electrical",
    "GarageType",
    "PavedDrive",
    "MiscFeature",
    "SaleType",
    "SaleCondition",
]

In [ ]:
# Preprocesador Compuesto: Inputación + Escalamiento/Codificación.

# Pipeline Numerica
num_pipe = Pipeline(
    steps=[
        ("imputer_num", KNNImputer(n_neighbors=2, weights="uniform")),
        ("scaler", StandardScaler()),
    ]
)

# Pipeline categorica
cat_pipe = Pipeline(
    steps=[
        ("imputer_cat", SimpleImputer(strategy="constant", fill_value="missing")),
        ("onehot", OneHotEncoder(sparse=False, handle_unknown="ignore")),
    ]
)

# Pipeline Ordinal
ord_pipe = Pipeline(
    steps=[
        ("imputer_ord", SimpleImputer(strategy="constant", fill_value="NA")),
        ("ordinal", OrdinalEncoder(categories=ordinal_cat)),
    ]
)

# Preprocesador Compuesto
prep = ColumnTransformer(
    transformers=[
        ("num", num_pipe, num_cols),
        ("cat", cat_pipe, cat_cols),
        ("ord", ord_pipe, ordinal_cols),
    ]
)

prep